# ARC TTT (Test-Time Training)

Ensemble solver with 3 solvers x 2 runs each (6 candidates total):
- **Perceiver**: Perception-based solver
- **Phased**: Structured tool-calling solver  
- **Iterative**: Code execution with iterative refinement

Voting + Judge selects top 2 submissions.

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

import json
import sys
import glob
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import numpy as np

KAGGLE = Path("/kaggle/input").exists()

if KAGGLE:
    ENSEMBLE_DIR = "/kaggle/input/arc-ttt/arc_ttt"
    EVAL_DIR = "/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges"
    OUTPUT_DIR = "/kaggle/working"
    
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        os.environ["OPENROUTER_API_KEY"] = user_secrets.get_secret("OPENROUTER_API_KEY")
        print("✓ API key loaded from Kaggle secrets")
    except Exception as e:
        print(f"⚠️ Failed to load API key from secrets: {e}")
        os.environ["OPENROUTER_API_KEY"] = ""
else:
    ENSEMBLE_DIR = "/path/to/arc_ttt"
    EVAL_DIR = "/path/to/arc/evaluation"
    OUTPUT_DIR = ENSEMBLE_DIR
    
    os.environ["OPENROUTER_API_KEY"] = os.environ.get("OPENROUTER_API_KEY", "YOUR_API_KEY_HERE")

if not os.environ.get("OPENROUTER_API_KEY"):
    raise ValueError("OPENROUTER_API_KEY not found. Set it as an environment variable or use Kaggle secrets.")

sys.path.insert(0, ENSEMBLE_DIR)
os.chdir(ENSEMBLE_DIR)

from orchestrator import solve_with_judge, solve_batch_with_judge
from config import reset_usage_stats, get_usage_stats

print("=" * 60)
print("🚀 ARC TTT SOLVER")
print("=" * 60)
print(f"Environment: {'Kaggle' if KAGGLE else 'Local'}")
print(f"Ensemble Dir: {ENSEMBLE_DIR}")
print(f"Eval Dir: {EVAL_DIR}")
print(f"Output Dir: {OUTPUT_DIR}")

In [ ]:
MAX_CONCURRENT = 60

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RESULTS_FILE = os.path.join(OUTPUT_DIR, f"results_{timestamp}.jsonl")
SUBMISSION_FILE = os.path.join(OUTPUT_DIR, f"submission_{timestamp}.json")

print(f"Results file: {RESULTS_FILE}")
print(f"Submission file: {SUBMISSION_FILE}")

submission_data = {}
FALLBACK = [[0, 0], [0, 0]]

In [ ]:
def load_task(task_id: str) -> dict:
    task_path = os.path.join(EVAL_DIR, f"{task_id}.json")
    with open(task_path) as f:
        return json.load(f)

def get_ground_truth_from_task(task_data: dict) -> list[np.ndarray]:
    outputs = []
    for test in task_data.get('test', []):
        if 'output' in test:
            outputs.append(np.array(test['output']))
    return outputs if outputs else None

all_task_files = glob.glob(os.path.join(EVAL_DIR, "*.json"))
all_task_ids = [os.path.basename(f).replace(".json", "") for f in all_task_files]

print(f"Found {len(all_task_ids)} tasks in evaluation directory")

In [ ]:
TASKS = all_task_ids

print(f"Running on {len(TASKS)} tasks")

In [ ]:
def check_attempt(attempt, ground_truths: list[np.ndarray]) -> tuple[bool, int, int]:
    if attempt is None or ground_truths is None or len(ground_truths) == 0:
        return (False, 0, len(ground_truths) if ground_truths else 0)
    
    n_tests = len(ground_truths)
    
    try:
        if n_tests == 1:
            attempt_arr = np.array(attempt)
            gt = ground_truths[0]
            correct = attempt_arr.shape == gt.shape and np.array_equal(attempt_arr, gt)
            return (correct, 1 if correct else 0, 1)
        else:
            is_list_of_grids = (
                isinstance(attempt, list) and 
                len(attempt) > 0 and 
                isinstance(attempt[0], list) and 
                len(attempt[0]) > 0 and
                isinstance(attempt[0][0], list)
            )
            
            if not is_list_of_grids:
                attempt_arr = np.array(attempt)
                gt = ground_truths[0]
                correct = attempt_arr.shape == gt.shape and np.array_equal(attempt_arr, gt)
                return (False, 1 if correct else 0, n_tests)
            else:
                tests_correct = 0
                for i, gt in enumerate(ground_truths):
                    if i < len(attempt):
                        attempt_arr = np.array(attempt[i])
                        gt_arr = np.array(gt)
                        if attempt_arr.shape == gt_arr.shape and np.array_equal(attempt_arr, gt_arr):
                            tests_correct += 1
                return (tests_correct == n_tests, tests_correct, n_tests)
    except Exception:
        return (False, 0, n_tests)


def calculate_task_score(result: dict, ground_truths: list[np.ndarray]) -> tuple[float, float, float]:
    if ground_truths is None or len(ground_truths) == 0:
        return (0.0, 0.0, 0.0)
    
    attempt_1 = result.get("attempt_1")
    attempt_2 = result.get("attempt_2")
    
    _, tests_correct_1, total_tests = check_attempt(attempt_1, ground_truths)
    _, tests_correct_2, _ = check_attempt(attempt_2, ground_truths)
    
    score_1 = tests_correct_1 / total_tests if total_tests > 0 else 0.0
    score_2 = tests_correct_2 / total_tests if total_tests > 0 else 0.0
    final_score = max(score_1, score_2)
    
    return (score_1, score_2, final_score)

## Run Tasks

In [ ]:
import gc

reset_usage_stats()
results = []
total_score = 0
score_1_total = 0
score_2_total = 0
partial_count = 0  
completed = 0

source_counts = {"attempt_1": {}, "attempt_2": {}}
solver_wins = {"perceiver_1": 0, "perceiver_2": 0, "phased_1": 0, "phased_2": 0, "iterative_1": 0, "iterative_2": 0}

def update_submission_file(task_id, result, task_data):
    global submission_data
    
    n_tests = len(task_data.get('test', []))
    attempt_1 = result.get("attempt_1")
    attempt_2 = result.get("attempt_2")
    
    preds = []
    for i in range(n_tests):
        if n_tests == 1:
            a1 = attempt_1 if attempt_1 is not None else FALLBACK
            a2 = attempt_2 if attempt_2 is not None else FALLBACK
        else:
            if isinstance(attempt_1, list) and len(attempt_1) > i and isinstance(attempt_1[i], list):
                a1 = attempt_1[i]
            else:
                a1 = FALLBACK
            if isinstance(attempt_2, list) and len(attempt_2) > i and isinstance(attempt_2[i], list):
                a2 = attempt_2[i]
            else:
                a2 = FALLBACK
        preds.append({"attempt_1": a1, "attempt_2": a2})
    
    submission_data[task_id] = preds
    
    with open(SUBMISSION_FILE, 'w') as f:
        json.dump(submission_data, f)
    
    return len(submission_data)

task_cache = {}
gt_cache = {}

def process_task(task_id):
    task_data = load_task(task_id)
    gt = get_ground_truth_from_task(task_data)
    task_cache[task_id] = task_data
    gt_cache[task_id] = gt
    return solve_with_judge(task_data, task_id, gt, verbose=False)

with ThreadPoolExecutor(max_workers=MAX_CONCURRENT) as executor:
    futures = {executor.submit(process_task, tid): tid for tid in TASKS}
    
    for future in as_completed(futures):
        task_id = futures[future]
        try:
            result = future.result()
            result["task_id"] = task_id
            results.append(result)
            
            # Calculate score using cached ground truth
            gt = gt_cache.get(task_id) or get_ground_truth_from_task(load_task(task_id))
            score_1, score_2, task_score = calculate_task_score(result, gt)
            total_score += task_score
            score_1_total += score_1
            score_2_total += score_2
            
            # Track sources
            a1_src = result.get("attempt_1_source", "none")
            a2_src = result.get("attempt_2_source", "none")
            source_counts["attempt_1"][a1_src] = source_counts["attempt_1"].get(a1_src, 0) + 1
            source_counts["attempt_2"][a2_src] = source_counts["attempt_2"].get(a2_src, 0) + 1
            
            # Track which solver won
            if score_1 == 1.0:
                solver_wins[a1_src] = solver_wins.get(a1_src, 0) + 1
            elif score_2 == 1.0:
                solver_wins[a2_src] = solver_wins.get(a2_src, 0) + 1
            
            completed += 1
            
            scores_detail = result.get("scores", {})
            a1_scores = scores_detail.get("attempt_1", {})
            a2_scores = scores_detail.get("attempt_2", {})
            a1_partial = f" ({a1_scores.get('tests_correct', 0)}/{a1_scores.get('total_tests', 1)})" if a1_scores.get("partial") else ""
            a2_partial = f" ({a2_scores.get('tests_correct', 0)}/{a2_scores.get('total_tests', 1)})" if a2_scores.get("partial") else ""
            
            if 0 < task_score < 1.0:
                partial_count += 1
            
            if task_score == 1.0:
                status = "✓ PASS"
            elif task_score > 0:
                status = "⚠ PARTIAL"
            else:
                status = "✗ FAIL"
            print(f"[{completed}/{len(TASKS)}] {task_id}: {status} (s1={score_1:.2f} s2={score_2:.2f} best={task_score:.2f}) | a1={a1_src} a2={a2_src}")
            
            with open(RESULTS_FILE, "a") as f:
                f.write(json.dumps({
                    "task_id": task_id,
                    "attempt_1": result.get("attempt_1"),
                    "attempt_2": result.get("attempt_2"),
                    "attempt_1_source": a1_src,
                    "attempt_2_source": a2_src,
                    "ratings": [(r[0], r[1], r[2][:100] if len(r) > 2 and r[2] else "") for r in result.get("ratings", [])],
                    "score_1": score_1,
                    "score_2": score_2,
                    "final_score": task_score,
                    "scores": result.get("scores", {}),  # Detailed scores with tests_correct, total_tests, partial
                    "num_distinct": result.get("num_distinct", 0),
                    "total_candidates": result.get("total_candidates", 6),
                    "distinct_outputs": result.get("distinct_outputs", []),
                    "judge_used": result.get("judge_used", False),
                    "timing": result.get("timing"),
                    "usage": result.get("usage"),
                }) + "\n")
            
            # Update submission file (auto-updates as each task completes)
            task_data = task_cache.get(task_id) or load_task(task_id)
            n_submitted = update_submission_file(task_id, result, task_data)
            print(f"    💾 Submission updated: {n_submitted} tasks")
            
            # Flush memory after each task
            if 'all_outputs' in result:
                del result['all_outputs']  # Large object no longer needed
            if task_id in task_cache:
                del task_cache[task_id]  # Clear cached task data
            gc.collect()
            
        except Exception as e:
            import traceback
            print(f"[{completed}/{len(TASKS)}] {task_id}: ERROR - {e}")
            traceback.print_exc()
            completed += 1

print(f"\n{'='*60}")
print(f"FINAL RESULTS")
print(f"{'='*60}")
print(f"  Score 1 (Attempt 1): {score_1_total:.2f}/{len(TASKS)} = {score_1_total/len(TASKS)*100:.1f}%")
print(f"  Score 2 (Attempt 2): {score_2_total:.2f}/{len(TASKS)} = {score_2_total/len(TASKS)*100:.1f}%")
print(f"  Combined Score:      {total_score:.2f}/{len(TASKS)} = {total_score/len(TASKS)*100:.1f}%")
print(f"  Partial Tasks:       {partial_count}/{len(TASKS)} (tasks with 0 < score < 1)")
print(f"\n  Solver Win Counts:")
for solver, wins in sorted(solver_wins.items(), key=lambda x: -x[1]):
    print(f"    {solver}: {wins}")
print(f"\n  Attempt 1 Sources:")
for src, cnt in sorted(source_counts["attempt_1"].items(), key=lambda x: -x[1]):
    print(f"    {src}: {cnt}")
print(f"\n  Attempt 2 Sources:")
for src, cnt in sorted(source_counts["attempt_2"].items(), key=lambda x: -x[1]):
    print(f"    {src}: {cnt}")
print(f"\n{'='*60}")
print(get_usage_stats())